In [2]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [4]:
# df = pd.read_csv('./netflix-prize-data/movie_titles.csv', header=None, delimiter="\t", encoding = "ISO-8859-1")

# train_df1 = pd.read_table('./netflix-prize-data/combined_data_1.txt', header=None)

In [1]:
# df = train_df1.sample(frac=1.0)  # 全部打乱
# cut_idx = int(round(0.1 * df.shape[0]))
# df_train = df.iloc[:cut_idx]
# print(df.shape, df_train.shape)  # (3184, 12) (318, 12) (2866, 12)|

In [6]:
with open("./netflix-prize-data/combined_data_1.txt", "r") as f:  # 打开文件
    data = f.readlines()  # 读取文件

In [17]:
data_array = []
movieId = ''
for i in range(int(len(data)/ 5)):
    if ':' in data[i]:
        movieId = data[i].split(":")[0]
        continue
    customerID = data[i].split(",")[0]
    rating = data[i].split(",")[1]
    data_array.append([customerID, movieId, rating])

In [18]:
data_df = pd.DataFrame(data_array,columns=['customerID','movieId','rating'])

In [24]:
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).
dataTrain = Dataset.load_from_df(data_df[['customerID','movieId','rating']], reader)

In [25]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), dataTrain, cv=2)

{'test_rmse': array([1.44886832, 1.44970245]),
 'test_mae': array([1.16083728, 1.16181473]),
 'fit_time': (8.39664602279663, 12.16157841682434),
 'test_time': (40.111706018447876, 45.42388129234314)}

In [29]:
# We can now u                        se this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), dataTrain, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.95500535, 0.95428015, 0.95416348, 0.95614176, 0.9547556 ]),
 'test_mae': array([0.75840502, 0.7577454 , 0.75709479, 0.75876729, 0.7582852 ]),
 'fit_time': (30.31313705444336,
  37.759105920791626,
  37.86931753158569,
  34.17902159690857,
  38.02493381500244),
 'test_time': (78.31344270706177,
  8.524463891983032,
  66.32598209381104,
  6.840270042419434,
  15.304190158843994)}

In [27]:
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold

train_set = dataTrain.build_full_trainset()

# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}

algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(dataTrain):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
RMSE: 0.9560
Estimating biases using als...
RMSE: 0.9561
Estimating biases using als...
RMSE: 0.9560
